# XBNet

‘XBNet’, which stands for ‘Extremely Boosted Neural Network’, that combines gradient boosted tree with a feed-forward neural network, making the model robust for all performance metrics. In this approach, trees are being trained in every layer of the architecture, and feature importance given by the trees and weight determined by gradient descent is used to adjust the weights of layers where trees are trained.      

XBNet takes raw tabular data as input, and the model is being trained using an optimization technique called Boosted Gradient Descent which is initialized with the help of feature importance of a gradient boosted trees further it updates the weights of each layer in the neural network in two steps as below:

> * Updates Weight by Gradient descent.
> * Updates weights by using feature importance of gradient boosted trees.  

To read about it more, please refer [this](https://analyticsindiamag.com/guide-to-xbnet-an-extremely-boosted-neural-network/) article.

# Code Implementation

Here we will compare the performance of XBNet and custom neural networks maintaining the same training parameters.
XBNet:

Install the architecture using pip as below

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy sklearn statsmodels keras tensorflow torch --user -q

In [ ]:
!python -m pip install --upgrade git+https://github.com/tusharsarkar3/XBNet.git --user -q --no-warn-script-location

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

Import all the dependencies:

In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from XBNet.training_utils import training,predict
from XBNet.models import XBNETClassifier
from XBNet.run import run_XBNET

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
data = load_iris()

In [ ]:
data.target

Set the input output features and train test split

In [ ]:
x = data.data
y = data.target
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.3, random_state= True)

Initialize the architecture with training data; while initializing you need to set input-output dimensions of each layer here. I have set the number of layers as two, so I need to set the dimension manually. Don’t worry; it is pretty straightforward; you will be prompted to do so, as shown below.  

In [ ]:
model = XBNETClassifier(x_train,y_train,num_layers=2)

Set the loss function and optimizer.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Run the architecture using run_XBNET

In [ ]:
m,acc, lo, val_ac, val_lo = run_XBNET(x_train,x_test,y_train,y_test,model,criterion,optimizer,epochs=100,batch_size=32)

Classification report for training and validation, respectively. 

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(acc,label='training accuracy')
plt.plot(val_ac,label = 'validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(lo,label='training loss')
plt.plot(val_lo,label = 'validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

##Custom neural network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv1D,Flatten

In [ ]:
model2 = Sequential()

In [ ]:
model2=Sequential()
model2.add(Conv1D(30,3, input_shape=(4,1), activation='relu'))                                        ## created simple neural network
model2.add(Dense(10,activation='relu'))  
model2.add(Flatten())                                                                 ## added hiden layer
model2.add(Dense(3,activation='softmax'))

In [ ]:
model2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
x=data.data
y=data.target.reshape(-1,1)
y=OneHotEncoder().fit_transform(y).toarray()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=True,test_size=0.33)

In [ ]:
x_train = x_train.reshape(100, 4,1)
x_test = x_test.reshape(50, 4,1)

In [ ]:
history = model2.fit(x_train,y_train,epochs=50,validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'],label='training accuracy')
plt.plot(history.history['val_accuracy'],label = 'validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'],label='training loss')
plt.plot(history.history['val_loss'],label = 'validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,model2.predict(x_test).round()))

In [ ]:
print(classification_report(y_train,model2.predict(x_train).round()))